In [1]:
import SALPY_LinearStage
import SALPY_Electrometer
import SALPY_ATMonochromator
import SALPY_FiberSpectrograph
import SALPY_ATSpectrograph
from lsst.ts import salobj
import asyncio

Could not import lsstcppimport; please ensure the base package has been built (not just setup).



In [164]:
ls_1_remote = salobj.Remote(SALPY_LinearStage,1)
ls_2_remote = salobj.Remote(SALPY_LinearStage,2)
electrometer_remote = salobj.Remote(SALPY_Electrometer,1)
monochromator_remote = salobj.Remote(SALPY_ATMonochromator)
fiber_spectrograph_remote = salobj.Remote(SALPY_FiberSpectrograph)
at_spectrograph_remote = salobj.Remote(SALPY_ATSpectrograph)

In [3]:
async def cmd(cbp_remote,name,parameters=[]):
    cmd_topic = getattr(cbp_remote,f"cmd_{name}").DataType()
    for parameter in parameters:
        setattr(cmd_topic,parameter[0],parameter[1])
    cmd_ack = await getattr(cbp_remote,f"cmd_{name}").start(cmd_topic,timeout=10)

In [16]:
start = await cmd(ls_2_remote,"start")

In [17]:
enable = await cmd(ls_2_remote,"enable")

In [44]:
get_home_1 = await cmd(ls_1_remote,"getHome")
get_home_2 = await cmd(ls_2_remote,"getHome")

In [158]:
await cmd(electrometer_remote,"")

Task exception was never retrieved
future: <Task finished coro=<_CommandInfo._start_timeout() done, defined at /home/saluser/repos/ts_salobj/python/lsst/ts/salobj/topics/remote_command.py:99> exception=CancelledError()>
concurrent.futures._base.CancelledError


AckError: Command failed with ack code -301

In [ ]:
ss = await monochromator_remote.evt_summaryState.next(flush=True)

In [162]:
try:
    await cmd(monochromator_remote,"enable")
except salobj.AckError as ae:
    print(ae.ack.result)

Task exception was never retrieved
future: <Task finished coro=<_CommandInfo._start_timeout() done, defined at /home/saluser/repos/ts_salobj/python/lsst/ts/salobj/topics/remote_command.py:99> exception=CancelledError()>
concurrent.futures._base.CancelledError


In [165]:
try:
    await cmd(monochromator_remote,"standby")
except salobj.AckError as ae:
    print(ae.ack.result)

Failed: standby not allowed in state <State.ENABLED: 2>


In [113]:
try:
    coro = monochromator_remote.evt_summaryState.next(flush=True)
    await cmd(monochromator_remote,"enable")
    ss = await coro
    print(salobj.State(ss.summaryState))
except salobj.AckError as ae:
    print(ae.ack.result)
    

In [153]:
try:
    await cmd(monochromator_remote,"standby")
except salobj.AckError as ae:
    print(ae.ack.result)

In [148]:
try:
    await cmd(monochromator_remote,"exitControl")
except salobj.AckError as ae:
    print(ae.ack.result)

In [18]:
async def evt(remote,name):
    evt_ack = await getattr(remote,f"evt_{name}").next(flush=True)

In [143]:
s=monochromator_remote.evt_summaryState.get()
e=monochromator_remote.evt_errorCode.get()
lm=monochromator_remote.evt_logMessage.get()

In [144]:
salobj.State(s.summaryState)

<State.STANDBY: 5>

In [145]:
print(e)

None


In [146]:
print(lm.message)

coro cmd_enable callback failed


In [121]:
monochromator_remote.cmd_changeWavelength.set(wavelength=400)
e = await monochromator_remote.cmd_changeWavelength.start()